In [31]:
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.model_selection import train_test_split
import lightgbm as lgbm
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import math
% matplotlib inline

In [2]:
raw_train_data=pd.read_csv("../../../Users/sreek/Documents/wer_ai/case-studies/train.tsv",delimiter='\t')


 Rows where numerical values ar NAN will be imputed while rows where
categorical values are missing will be dropped.Dropping  rows where columns f_61,f_121,f_215, f_237 have null values.

In [3]:
raw_train_dropped=raw_train_data.dropna(subset=['f_61','f_121','f_215','f_237'])

In [ ]:
raw_train_dropped.shape

In [4]:
del raw_train_data

In [5]:
test_data=pd.read_csv("../../../Users/sreek/Documents/wer_ai/case-studies/test.tsv",delimiter='\t')

In [5]:
target_data=raw_train_dropped.target

In [6]:
raw_train_dropped.drop('target',axis=1,inplace=True)

# numerical features

In [7]:
X_train_num=raw_train_dropped.loc[:,raw_train_dropped.dtypes =='float64']

# categorical features

In [9]:
X_train_cat=raw_train_dropped.loc[:,raw_train_dropped.dtypes == object]

In [10]:
cat_features=X_train_cat.columns

Hashing categorical columns

In [11]:
def feature_hash(df, max_size=20):
    for col in cat_features:
        df[col+"_hash"] = df[col].apply(lambda x: hash(x)%max_size)
    return df

In [12]:
X_train_cat_new=feature_hash(X_train_cat)

C:\python\pyLibs\Anaconda\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [13]:
X_train_cat_new.drop(['f_61','f_121','f_215','f_237'],axis=1,inplace=True)

C:\python\pyLibs\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
X_train_cat_scaled=StandardScaler().fit_transform(X_train_cat_new.values)

C:\Users\sreek\AppData\Roaming\Python\Python35\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\sreek\AppData\Roaming\Python\Python35\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# Imputing missing numerical values

In [ ]:
imputer=Imputer()

In [15]:
X_train_num_imputed=imputer.fit_transform(X_train_num.values)

In [16]:
X_train_num_scaled=StandardScaler().fit_transform(X_train_num_imputed)

In [17]:
m_train=np.hstack([X_train_num_scaled,X_train_cat_scaled])

In [18]:
#pca=PCA(n_components=0.95)

In [19]:
#m_train_reduced=pca.fit_transform(m_train)

In [20]:
#np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

array([ 0.75,  1.48,  2.21,  2.91,  3.6 ,  4.19,  4.77,  5.35,  5.92,
        6.49,  7.05,  7.61,  8.17,  8.73,  9.28,  9.83, 10.38, 10.92,
       11.46, 12.  , 12.54, 13.08, 13.61, 14.14, 14.67, 15.2 , 15.73,
       16.26, 16.78, 17.3 , 17.82, 18.34, 18.85, 19.36, 19.87, 20.38,
       20.89, 21.4 , 21.91, 22.42, 22.92, 23.42, 23.92, 24.42, 24.92,
       25.42, 25.91, 26.4 , 26.89, 27.38, 27.87, 28.36, 28.84, 29.32,
       29.8 , 30.28, 30.76, 31.23, 31.7 , 32.17, 32.64, 33.11, 33.58,
       34.05, 34.51, 34.97, 35.43, 35.89, 36.35, 36.81, 37.27, 37.72,
       38.17, 38.62, 39.07, 39.52, 39.97, 40.42, 40.87, 41.31, 41.75,
       42.19, 42.63, 43.07, 43.51, 43.95, 44.38, 44.81, 45.24, 45.67,
       46.1 , 46.53, 46.96, 47.39, 47.82, 48.24, 48.66, 49.08, 49.5 ,
       49.92, 50.34, 50.75, 51.16, 51.57, 51.98, 52.39, 52.8 , 53.21,
       53.62, 54.02, 54.42, 54.82, 55.22, 55.62, 56.02, 56.42, 56.82,
       57.22, 57.62, 58.02, 58.41, 58.8 , 59.19, 59.58, 59.97, 60.36,
       60.75, 61.13,

In [45]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'eval_metric': 'rmse',
    'max_depth': 17, 
    'learning_rate': 0.10,
    'verbose': 0 
    }
n_estimators = 100


In [48]:
from sklearn.metrics import explained_variance_score
n_iters = 5
for i in range(n_iters): 
    x_train, x_valid, y_train, y_valid = train_test_split(m_train, target_data, test_size=0.15, random_state=i)
    d_train = lgbm.Dataset(x_train, label=y_train)
    d_valid = lgbm.Dataset(x_valid, label=y_valid)
    watchlist = [d_valid]

    model = lgbm.train(params, d_train, n_estimators,valid_sets=[d_train,d_valid], early_stopping_rounds=20, verbose_eval=1)

    preds = model.predict(x_valid)
    
  

#print('rmse of prediction is:', mean_squared_error(y_valid, preds) ** 0.5)  


print("RMSE of preds in ",explained_variance_score(y_valid,preds))
print(model.feature_importance())



[1]	training's l2: 24.5036	valid_1's l2: 27.9392
Training until validation scores don't improve for 20 rounds.
[2]	training's l2: 22.3058	valid_1's l2: 25.7062
[3]	training's l2: 20.4068	valid_1's l2: 23.8014
[4]	training's l2: 18.8805	valid_1's l2: 22.3022
[5]	training's l2: 17.5646	valid_1's l2: 21.0486
[6]	training's l2: 16.4266	valid_1's l2: 20.0041
[7]	training's l2: 15.2652	valid_1's l2: 19.0002
[8]	training's l2: 14.4154	valid_1's l2: 18.1007
[9]	training's l2: 13.5965	valid_1's l2: 17.1685
[10]	training's l2: 12.9471	valid_1's l2: 16.6614
[11]	training's l2: 12.1685	valid_1's l2: 15.7766
[12]	training's l2: 11.5662	valid_1's l2: 15.2133
[13]	training's l2: 11.0297	valid_1's l2: 14.7169
[14]	training's l2: 10.43	valid_1's l2: 14.2144
[15]	training's l2: 10.0212	valid_1's l2: 13.8557
[16]	training's l2: 9.50336	valid_1's l2: 13.3873
[17]	training's l2: 9.09521	valid_1's l2: 13.0295
[18]	training's l2: 8.5833	valid_1's l2: 12.4161
[19]	training's l2: 8.25614	valid_1's l2: 12.1535
